# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set our OpenAI, Tavily, and LangSmith API keys along with our LangSmith environment variables.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

TAVILY_API_KEY ········


In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE8 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

LangSmith API Key:  ········


## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

C:\Users\mspla\AppData\Local\Temp\ipykernel_18204\1203815797.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?


The model is given tool specs (name/description/JSON schema) via bind_tools, reasons over the prompt + state, emits a tool_call when helpful, LangGraph routes to a ToolNode to execute it, adds the observation back into state, and repeats until the model chooses to finalize.


1. Tools are registered & bound
Each tool is given a name, description, and JSON schema (args). In LangChain you do this with bind_tools(...). These specs are sent to the model as the tools array so the LLM can “see” what’s available.

2. Model reasons with the current state
The prompt (system + chat history + any state) instructs the LLM to answer directly if it can; otherwise consider tools. This is a learned, context-aware choice—no hardcoded if/else.

3. LLM emits a tool call when needed
If the model decides a tool is useful, it returns a tool_call (function name + JSON args) rather than a final answer.

4. Graph routes to the Tool node
LangGraph uses a conditional edge:
if tool_calls present → go to ToolNode to execute;
else → END (final answer).

5. Observation → loop back → synthesize
The ToolNode executes the function, appends the observation to state/messages, and the graph loops back to the model. The model then uses that observation to answer or to call another tool. Loop ends when the model stops emitting tool calls (or a guard/limit triggers).





## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [9]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [10]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [11]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [12]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [13]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [15]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

There isn’t a limit number on cycles. LangGraph enforces a recursion limit (default ~25 steps) to prevent infinite loops; hitting it raises GraphRecursionError. You can configure this via recursion_limit per run. 

If not, how could we impose a limit to the number of cycles?
Practically, keep iterations low for traceability and cost, and add interrupts/guards or return_direct to stop earlier when the answer is known. 3-4 cycles or less , as a rule of thumb.


## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [16]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="How are technical professionals using AI to improve their work?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")


Receiving update from node: 'agent'
[AIMessage(content='Technical professionals are using AI in various ways to enhance their work, including automating repetitive tasks, improving data analysis, developing smarter software, and creating innovative solutions. Would you like me to provide specific examples or recent trends in AI applications across different technical fields?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 163, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CKlSbWiUu6RScBCAMmQ2fLANes52D', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d046bcd4-54fe-49e8-bbc2-1f23b0aba69e-0', usage_metadata={'input_to

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [17]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the A Comprehensive Survey of Deep Research paper, then search each of the authors to find out where they work now using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bbiAPEuZKzB4yKc73XfhF4VG', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_JvcInpcTnlhn7kM2rtsPvH8b', 'function': {'arguments': '{"query": "A Comprehensive Survey of Deep Research paper"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 182, 'total_tokens': 241, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CKlSh3LH1RDhdv19icfr42L7ivWI4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--72e040fa-e926-4f

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

Human → agent ──(tool_calls: arxiv, tavily)──► router ─► action (tools run)
▲ │
└───────────── agent ◄──────────────┘
(final AIMessage, no tool_calls)
router → END



Steps the agent took:
State initialised: the user’s query is added to state["messages"] as a HumanMessage.
Agent analyses and plans: the agent node runs the LLM, decides it needs external info, and emits an AIMessage with tool_calls to ArXiv {"query": "A Comprehensive Survey of Deep Research"} and Tavily {"query": "author of A Comprehensive Survey of Deep Research"}. The AIMessage content is empty on a tool-call turn and finish_reason is "tool_calls".
Router decision: the conditional edge inspects the last AIMessage, finds tool_calls, and routes to the action (tool) node.
Action executes tools: the action node runs both tools (usually sequentially), appends ToolMessages with results to state["messages"], and returns control to the agent.
Agent synthesises answer: the agent reads the tool outputs and composes a final AIMessage without tool_calls.
Router finalises: the router checks the last AIMessage, finds no tool_calls, routes to END, and the final answer is emitted.

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [18]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["text"])]}

def parse_output(input_state):
  return {"answer" : input_state["messages"][-1].content}

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

agent_chain_with_formatting.invoke({"text" : "What is Deep Research?"})

{'answer': 'Deep Research is an advanced AI-powered research tool designed to conduct in-depth, multi-step investigations on the Internet. Unlike standard chatbots that provide quick, surface-level responses, Deep Research autonomously finds, analyzes, and synthesizes information from hundreds of online sources to produce comprehensive and reliable reports. It is particularly useful for complex research tasks in areas such as science, finance, policy, and engineering, where thorough and precise information gathering is essential. Deep Research can interpret and analyze text, images, and PDFs, and it organizes the gathered information into structured, citeable outputs. It was developed by OpenAI and is integrated into ChatGPT, offering a powerful solution for detailed research and content creation.'}

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    {
        "inputs" : {"text" : "Who were the main authors on the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' paper?"},
        "outputs" : {"must_mention" : ["Peng", "Xu"]}   
    },
    ...,
    {
        "inputs" : {"text" : "Where do the authors of the 'A Comprehensive Survey of Deep Research: Systems, Methodologies, and Applications' work now?"},
        "outputs" : {"must_mention" : ["Zhejiang", "Liberty Mutual"]}
    }
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions that pertain to the cohort use-case (more information [here](https://www.notion.so/Session-4-RAG-with-LangGraph-OSS-Local-Models-Eval-w-LangSmith-26acd547af3d80838d5beba464d7e701#26acd547af3d81d08809c9c82a462bdd)), or the use-case you're hoping to tackle in your Demo Day project.

In [34]:
questions = [
    {
        "inputs": {
            "text": "A 58-year-old with crushing chest pain, ST-elevations in V2–V4, and elevated troponin—classify the concept 'Acute anterior myocardial infarction' by SNOMED CT highest-level category."
        },
        "outputs": {
            "ideal_answer": "Clinical finding",
            "must_mention": ["Clinical finding"]
        }
    },
    {
        "inputs": {
            "text": "A child develops itchy wheals minutes after eating peanuts; the clinician records 'Urticaria'—assign the SNOMED CT highest-level category."
        },
        "outputs": {
            "ideal_answer": "Clinical finding",
            "must_mention": ["Clinical finding"]
        }
    },
    {
        "inputs": {
            "text": "A patient with suspected pulmonary embolism undergoes 'CT pulmonary angiography'—classify this by SNOMED CT highest-level category."
        },
        "outputs": {
            "ideal_answer": "Procedure",
            "must_mention": ["Procedure"]
        }
    },
    {
        "inputs": {
            "text": "For streptococcal pharyngitis, the pharmacy dispenses 'Amoxicillin 500 mg capsule'—assign the SNOMED CT highest-level category."
        },
        "outputs": {
            "ideal_answer": "Pharmaceutical / biologic product",
            "must_mention": ["Pharmaceutical / biologic product"]
        }
    },
    {
        "inputs": {
            "text": "MRI confirms a tear of the anterior cruciate ligament; classify the concept 'Anterior cruciate ligament of knee' by SNOMED CT highest-level category."
        },
        "outputs": {
            "ideal_answer": "Body structure",
            "must_mention": ["Body structure"]
        }
    }
]


Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [35]:
from langsmith import Client

client = Client()

dataset_name = f"Simple Search Agent - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the medical use-case to evaluate the Simple Search Agent."
)

client.create_examples(
    dataset_id=dataset.id,
    examples=questions
)

{'example_ids': ['ca435b92-e6ad-4838-8032-af109920ecb8',
  '44b6a6a1-28d8-4a87-a426-7ecdef8e128f',
  '0c7f1a12-ff74-415e-9798-365f8f00d71b',
  'f53408e9-34c3-4309-9cf1-4a1a277a7189',
  '3c9b97cd-3189-455c-8ac8-4d83e8db6e8d'],
 'count': 5}

### Task 2: Adding Evaluators

Let's use the OpenEvals library to product an evaluator that we can then pass into LangSmith!

> NOTE: Examine the `CORRECTNESS_PROMPT` below!

In [43]:
from openevals.prompts import CORRECTNESS_PROMPT
print(CORRECTNESS_PROMPT)

You are an expert data labeler evaluating model outputs for correctness. Your task is to assign a score based on the following rubric:

<Rubric>
  A correct answer:
  - Provides accurate and complete information
  - Contains no factual errors
  - Addresses all parts of the question
  - Is logically consistent
  - Uses precise and accurate terminology

  When scoring, you should penalize:
  - Factual errors or inaccuracies
  - Incomplete or partial answers
  - Misleading or ambiguous statements
  - Incorrect terminology
  - Logical inconsistencies
  - Missing key information
</Rubric>

<Instructions>
  - Carefully read the input and output
  - Check for factual accuracy and completeness
  - Focus on correctness of information rather than style or verbosity
</Instructions>

<Reminder>
  The goal is to evaluate factual correctness and completeness of the response.
</Reminder>

<input>
{inputs}
</input>

<output>
{outputs}
</output>

Use the reference outputs below to help you evaluate the

In [44]:
from openevals.llm import create_llm_as_judge

correctness_evaluator = create_llm_as_judge(
        prompt=CORRECTNESS_PROMPT,
        model="openai:o3-mini", # very impactful to the final score
        feedback_key="correctness",
    )

Let's also create a custom Evaluator for our created dataset above - we do this by first making a simple Python function!

In [45]:
def must_mention(inputs: dict, outputs: dict, reference_outputs: dict) -> float:
  # determine if the phrases in the reference_outputs are in the outputs
  required = reference_outputs.get("must_mention") or []
  score = all(phrase in outputs["answer"] for phrase in required)
  return score

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.
>
> As my case is in medical domain
> Immprove by providing guardrails for satety and potenttial for time senseitivy
> Prefer uncertainty over dangerous assumtpsion.
> Provide sample scores
>
example in cell below




In [42]:
MY_CORRECTNESS_PROMPT = '''
You are a clinical expert evaluating model outputs for factual and clinical correctness.

<Rubric>
A clinically correct answer MUST:
- Be medically accurate and complete for the question asked.
- State the correct SNOMED CT highest-level category (when applicable) and use precise clinical terminology.
- Be clinically safe: no harmful advice, no contraindicated actions, no unsafe dosing or routes.
- Align with recognised guidelines/evidence (state if evidence is uncertain or evolving).
- Address all parts of the question and provide an appropriate next step when relevant.
- Calibrate uncertainty (admit limits; avoid overconfident speculation).

Penalise for:
- Factual errors, category/ontology mistakes, missing key elements.
- Unsafe or contraindicated recommendations; dosing or unit errors.
- Out-of-date or non-evidence-based claims when recency matters.
- Overconfidence when evidence is weak; ambiguous or misleading wording.
</Rubric>

<Safety Sentinels (zero-tolerance; FAIL if any present)>
- Recommends contraindicated drug/procedure for the described patient.
- Provides dosing outside common therapeutic ranges without justification.
- Fabricates guideline citations or sources.
</Safety Sentinels>

<Scoring>
Score 0–5:
5 = Fully correct, safe, guideline-consistent, complete, calibrated.
4 = Correct and safe, minor omissions OR minor guideline/detail gap.
3 = Mostly correct but incomplete OR minor safety/terminology issues.
2 = Partly correct with notable gaps OR unclear safety stance.
1 = Largely incorrect or unsafe.
0 = Unsafe/harmful or mostly fabricated.
</Scoring>

<Instructions>
- Focus on clinical correctness, safety, and completeness over style.
- If the task is categorisation (e.g., SNOMED highest-level), ensure the category matches the ontology concept (accept exact synonyms).
- Prefer “uncertain / needs further workup” over unsafe speculation.
- If time-sensitive, flag when an answer may be outdated.
</Instructions>

<input>
{inputs}
</input>

<output>
{outputs}
</output>

Use the reference outputs below (gold answers, acceptable variants, or ontology IDs) to evaluate correctness:

<reference_outputs>
{reference_outputs}
</reference_outputs>
'''

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [47]:
results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset.name,
    evaluators=[correctness_evaluator, must_mention],
    experiment_prefix="simple_agent, baseline",  # optional, experiment name prefix
    description="Testing the baseline system.",  # optional, experiment description
    max_concurrency=4, # optional, add concurrency
)

View the evaluation results for experiment: 'simple_agent, baseline-78bfe6cd' at:
https://smith.langchain.com/o/3936adcd-6eec-4723-b49d-fee2168a2d46/datasets/1d923c1f-1c4e-4097-8aa8-1df50d8d7021/compare?selectedSessions=3f841899-8198-4dc3-82ba-5d322af943c8




0it [00:00, ?it/s]

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [48]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #4:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE
This cell initializes a new StateGraph called `graph_with_helpfulness_check` and adds the same two nodes:
- **"agent" node**: Calls the LLM model to generate responses
- **"action" node**: Executes tools when the agent decides to use them




In [50]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE
##### Setting the Entry Point
First node to be accessed.

In [51]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE
##### Creating Conditional Logic

This cell defines the `tool_call_or_helpful` function. This function:

1. **Checks for tool calls**: If the agent wants to use tools, it routes to the "action" node
2. **Implements loop limits**: If more than 10 messages have been exchanged, it stops to prevent infinite loops
3. **Evaluates helpfulness**: Uses a separate LLM to judge if the current response adequately answers the original question
4. **Makes routing decisions**: Returns "action", "continue", or "end" based on the analysis

This creates agent that can self-evaluate its responses and avoid getting stuck in loops.

In [52]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

##### YOUR MARKDOWN HERE
##### Connecting Nodes with Conditional Logic

This cell adds conditional edges from the "agent" node using our `tool_call_or_helpful` function. The routing logic works as follows:

- **"action"**: Routes to the tool execution node when tools are needed
- **"continue"**: Routes back to the agent node when the response isn't helpful enough
- **"end"**: Terminates the conversation when the response is satisfactory or loop limit is reached

This creates a dynamic flow that can cycle between agent and tools while maintaining quality control.

In [53]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE
##### Completing the Graph Structure

This cell adds the final edge that connects the "action" node back to the "agent" node. This ensures that after tools are executed, the agent can process the tool results and decide on the next step - whether to use more tools, provide a final answer, or continue the conversation.

This completes the cyclic structure of the graph.

In [54]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE
##### Compiling the Enhanced Agent

This cell compiles graph into a runnable agent. The compilation process:

1. **Validates the graph structure**: Ensures all nodes and edges are properly connected
2. **Optimizes execution**: Prepares the graph for efficient runtime performance
3. **Creates the runnable**: Produces an agent that can be invoked with user inputs

Our enhanced agent is now ready to use with built-in helpfulness checking and loop prevention!

In [55]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE
Demonstrate agent in action by asking it about Deep Research. The agent will:

1. **Analyze the question** and decide if tools are needed
2. **Use tools if necessary** to gather information
3. **Evaluate its response** for helpfulness
4. **Continue or stop** based on the evaluation

This shows how agent's ability to self-regulate and provide more thoughtful, complete responses while avoiding infinite loops.

In [56]:
inputs = {"messages" : [HumanMessage(content="What are Deep Research Agents?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='Deep Research Agents are advanced AI systems designed to assist with complex research tasks. They leverage deep learning techniques and large datasets to analyze, synthesize, and generate insights across various fields of study. These agents can automate literature reviews, identify relevant information, generate hypotheses, and even assist in experimental design. They are used in academia, industry, and research institutions to accelerate discovery and improve the accuracy and depth of research outcomes. Would you like more detailed information or specific examples of Deep Research Agents?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 158, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

## Part 3: LangGraph for the "Patterns" of GenAI

### Task 4: Helpfulness Check of Gen AI Pattern Descriptions

Let's ask our system about the 3 main patterns in Generative AI:

1. Context Engineering
2. Fine-tuning
3. Agents

In [57]:
patterns = ["Context Engineering", "Fine-tuning", "LLM-based agents"]

In [58]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Context Engineering is a relatively new interdisciplinary field that focuses on designing, managing, and optimizing the context in which systems, especially artificial intelligence and software applications, operate. It involves understanding and shaping the environment, circumstances, and background information that influence how systems behave and interact with users. The goal is to improve system performance, user experience, and decision-making by carefully controlling and leveraging context.

The concept of Context Engineering began gaining attention in the early 2020s, particularly with the rise of context-aware computing, personalized AI, and adaptive systems. It became more prominent as researchers and practitioners recognized the importance of context in creating more intelligent, responsive, and user-centric technologies.

Would you like me to find more detailed or specific information about its origins and development?



Fine-tuning is a machine learning technique used to a